In [1]:
library(caret, quiet=TRUE);
library(base64enc)
library(httr)



Attaching package: ‘httr’


The following object is masked from ‘package:caret’:

    progress




# Build a Model

In [2]:
set.seed(1960)

create_model  =  function() {

    # https://cran.r-project.org/web/packages/randomForest/randomForest.pdf
    model  <- train(Species ~ ., data = iris, method = "rf" , ntree=64)
    
    return(model)
}

In [3]:
# dataset
model = create_model()

In [4]:
pred <- predict(model, as.matrix(iris[, -5]) , type="prob")
pred_labels <- predict(model, as.matrix(iris[, -5]) , type="raw")
sum(pred_labels != iris$Species)/length(pred_labels)


[1] 0

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    # WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "caret_rpart_test_model", SerializedModel = b64_data , SQLDialect = "CPP" , Mode=" caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)

namespace  {

	namespace RF_Tree_0 {
	
		typedef std::vector<double> tNodeData;
		std::map<int, tNodeData> Decision_Tree_Node_data = {
				{ 1 ,  {1.0, 0.0, 0.0 }} ,
				{ 3 ,  {0.0, 1.0, 0.0 }} ,
				{ 6 ,  {0.0, 0.0, 1.0 }} ,
				{ 7 ,  {0.0, 1.0, 0.0 }} ,
				{ 10 ,  {0.0, 0.0, 1.0 }} ,
				{ 11 ,  {0.0, 0.0, 1.0 }} ,
				{ 12 ,  {0.0, 1.0, 0.0 }} 
		};
		
	
		int get_decision_tree_node_index(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {
			int lNodeIndex = (Feature_2 <= 2.45) ? ( 1 ) : ( (Feature_2 <= 4.75) ? ( 3 ) : ( (Feature_3 <= 1.7) ? ( (Feature_2 <= 4.95) ? ( 7 ) : ( (Feature_0 <= 6.05) ? ( (Feature_2 <= 5.05) ? ( 11 ) : ( 12 ) ) : ( 10 ) ) ) : ( 6 ) ) );
		
			return lNodeIndex;
		}
		
	
		std::vector<std::string> get_input_names(){
			std::vector<std::string> lFeatures = { "Feature_0", "Feature_1", "Feature_2", "Feature_3" };
	
			return lFeatures;
		}
	
		std::vector<std::any> get_classes(){
			std::vector<std::any> lClasses = { 0, 1, 2 

# Execute the CPP Code

In [7]:
    write_text_to_file = function(iCPPCode, oCPPFile) {
        fileConn<-file(oCPPFile)
        writeLines(c(iCPPCode), fileConn)
        close(fileConn)
    }

    add_cpp_main_function = function(iCPPCode, iCSVFile) {
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = paste(lCPPCode,  iCPPCode, sep="")
        lCPPCode = paste(lCPPCode, "\n\nint main() {\n", sep="")
        lCPPCode = paste(lCPPCode, "\tscore_csv_file(\"",  iCSVFile, "\");\n", sep="")
        lCPPCode = paste(lCPPCode, "\treturn 0;\n}\n", sep="")
        return(lCPPCode)
    }

    compile_cpp_code_as_executable = function(iName) {
        lCommand = paste("-Wall -Wno-unused-function -std=c++17 -g -o ", iName,  ".exe ",  iName,  ".cpp", sep="")
        cat(paste("EXECUTING " , "'",  lCommand,  "'", sep=""))
        result = system2("g++", args = lCommand)
        # print(result)
    }

    execute_cpp_model = function(iName, iCSVFile) {
        lOutName = paste(iName, ".out", sep="")
        result2 = system2(paste(iName, ".exe", sep="") ,  args = iCSVFile, stdout=lOutName)
        # cat(result2)
        return(result2)
    }
        
    execute_cpp_code = function(iCPPCode, iCSVFile) {
        lTimeStr = format(Sys.time(), "%Y-%m-%d_%H_%M_%S")
        lName = paste("/tmp/ml2cpp_r_", "sample_" , lTimeStr, sep="");
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, paste(lName, ".cpp", sep=""))
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        lOutName = paste(lName, ".out", sep="")
        return(lOutName)
    }

In [8]:
dataset = iris[,-5]

df_cpp_in = as.data.frame(dataset)
# cat(names(df_cpp_in))

names(df_cpp_in) = sprintf("Feature_%d",0:(ncol(df_cpp_in)-1))

write.csv(df_cpp_in, "/tmp/iris2.csv", row.names = FALSE, quote = FALSE)

lOutName = execute_cpp_code(lModelSQL, "/tmp/iris2.csv")
lOutName

EXECUTING '-Wall -Wno-unused-function -std=c++17 -g -o /tmp/ml2cpp_r_sample_2020-09-29_18_18_36.exe /tmp/ml2cpp_r_sample_2020-09-29_18_18_36.cpp'

[1] "/tmp/ml2cpp_r_sample_2020-09-29_18_18_36.out"

In [9]:
df_cpp_out <- read.csv(file = lOutName)
df_cpp_out$KEY = seq.int(nrow(df_cpp_out))
head(df_cpp_out)

,idx,Score_setosa,Score_versicolor,Score_virginica,Proba_setosa,Proba_versicolor,Proba_virginica,LogProba_setosa,LogProba_versicolor,LogProba_virginica,Decision,DecisionProba,KEY
,<int>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>
1,0,NA,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1,1
2,1,NA,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1,2
3,2,NA,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1,3
4,3,NA,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1,4
5,4,NA,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1,5
6,5,NA,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1,6


# R Caret RandomForest Output

In [10]:
pred_proba  =  predict(model, as.matrix(iris[,-5]), type = "prob")
df_r_out = data.frame(pred_proba)
names(df_r_out) = sprintf("Proba_%s",model$levels)

df_r_out$KEY = seq.int(nrow(df_r_out))
df_r_out$Score_setosa  =  NA
df_r_out$Score_versicolor  =  NA
df_r_out$Score_virginica  =  NA
df_r_out$LogProba_setosa  =  log(df_r_out$Proba_setosa)
df_r_out$LogProba_versicolor =  log(df_r_out$Proba_versicolor)
df_r_out$LogProba_virginica  =  log(df_r_out$Proba_virginica)
df_r_out$Decision =   predict(model, as.matrix(iris[,-5]), type = "raw")
df_r_out$DecisionProba =  apply(pred_proba, 1, function(x) max(x))
head(df_r_out)



,Proba_setosa,Proba_versicolor,Proba_virginica,KEY,Score_setosa,Score_versicolor,Score_virginica,LogProba_setosa,LogProba_versicolor,LogProba_virginica,Decision,DecisionProba
,<dbl>,<dbl>,<dbl>,<int>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
1,1,0,0,1,NA,NA,NA,0,-Inf,-Inf,setosa,1
2,1,0,0,2,NA,NA,NA,0,-Inf,-Inf,setosa,1
3,1,0,0,3,NA,NA,NA,0,-Inf,-Inf,setosa,1
4,1,0,0,4,NA,NA,NA,0,-Inf,-Inf,setosa,1
5,1,0,0,5,NA,NA,NA,0,-Inf,-Inf,setosa,1
6,1,0,0,6,NA,NA,NA,0,-Inf,-Inf,setosa,1


# Compare R and SQL output

In [11]:
df_merge = merge(x = df_r_out, y = df_cpp_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

,KEY,Proba_setosa_1,Proba_versicolor_1,Proba_virginica_1,Score_setosa_1,Score_versicolor_1,Score_virginica_1,LogProba_setosa_1,LogProba_versicolor_1,LogProba_virginica_1,⋯,Score_versicolor_2,Score_virginica_2,Proba_setosa_2,Proba_versicolor_2,Proba_virginica_2,LogProba_setosa_2,LogProba_versicolor_2,LogProba_virginica_2,Decision_2,DecisionProba_2
,<int>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,1,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1
2,2,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1
3,3,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1
4,4,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1
5,5,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1
6,6,1,0,0,NA,NA,NA,0,-Inf,-Inf,⋯,NA,NA,1,0,0,0,-Inf,-Inf,setosa,1


In [12]:
diffs_df = df_merge[df_merge$Decision_1 != df_merge$Decision_2,]
head(diffs_df)

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


KEY,Proba_setosa_1,Proba_versicolor_1,Proba_virginica_1,Score_setosa_1,Score_versicolor_1,Score_virginica_1,LogProba_setosa_1,LogProba_versicolor_1,LogProba_virginica_1,⋯,Score_versicolor_2,Score_virginica_2,Proba_setosa_2,Proba_versicolor_2,Proba_virginica_2,LogProba_setosa_2,LogProba_versicolor_2,LogProba_virginica_2,Decision_2,DecisionProba_2
<int>,<dbl>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>


In [13]:
stopifnot(nrow(diffs_df) == 0)

In [14]:
summary(df_cpp_out)

      idx         Score_setosa   Score_versicolor Score_virginica
 Min.   :  0.00   Mode:logical   Mode:logical     Mode:logical   
 1st Qu.: 37.25   NA's:150       NA's:150         NA's:150       
 Median : 74.50                                                  
 Mean   : 74.50                                                  
 3rd Qu.:111.75                                                  
 Max.   :149.00                                                  
  Proba_setosa    Proba_versicolor Proba_virginica  LogProba_setosa
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :-Inf   
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:-Inf   
 Median :0.0000   Median :0.0000   Median :0.0000   Median :-Inf   
 Mean   :0.3332   Mean   :0.3353   Mean   :0.3315   Mean   :-Inf   
 3rd Qu.:1.0000   3rd Qu.:0.9805   3rd Qu.:0.9844   3rd Qu.:   0   
 Max.   :1.0000   Max.   :1.0000   Max.   :1.0000   Max.   :   0   
 LogProba_versicolor LogProba_virginica   Decision         Dec

In [15]:
summary(df_r_out)

  Proba_setosa    Proba_versicolor Proba_virginica       KEY        
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :  1.00  
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.: 38.25  
 Median :0.0000   Median :0.0000   Median :0.0000   Median : 75.50  
 Mean   :0.3332   Mean   :0.3353   Mean   :0.3315   Mean   : 75.50  
 3rd Qu.:1.0000   3rd Qu.:0.9805   3rd Qu.:0.9844   3rd Qu.:112.75  
 Max.   :1.0000   Max.   :1.0000   Max.   :1.0000   Max.   :150.00  
 Score_setosa   Score_versicolor Score_virginica LogProba_setosa
 Mode:logical   Mode:logical     Mode:logical    Min.   :-Inf   
 NA's:150       NA's:150         NA's:150        1st Qu.:-Inf   
                                                 Median :-Inf   
                                                 Mean   :-Inf   
                                                 3rd Qu.:   0   
                                                 Max.   :   0   
 LogProba_versicolor LogProba_virginica       Decision  Decisi